In [ ]:
import numpy as np
import pandas as pd
import sqlite3

df = pd.read_csv("fashion_products.csv")


conn = sqlite3.connect('fashion_db.sqlite')
c = conn.cursor()


df.rename(columns={'User ID': 'user_id',
                   'Product ID': 'product_id',
                   'Product Name' : 'product_name',
                   'Brand' : 'brand',
                   'Category' : 'category',
                   'Price' : 'price',
                   'Color' : 'color',
                   'Size' : 'size'}, inplace=True)

c.execute('CREATE TABLE IF NOT EXISTS fashion_products (user_id int, product_id int, product_name text, brand text, category text, price int, rating float, color text, size text)')
conn.commit()

df.to_sql('fashion_products', conn, if_exists='replace', index = False)

c.execute('''
SELECT product_name FROM fashion_products LIMIT 100
          ''')

for row in c.fetchall():
    print (row)



In [3]:
def read_sql_query(sql, db):
    conn = sqlite3.connect(db)
    cur = conn.cursor()
    cur.execute(sql)
    rows = cur.fetchall()
    for row in rows:
        print(row)
    conn.close()

In [4]:
import sqlite3
read_sql_query('SELECT * FROM fashion_products LIMIT 10;',
               "fashion_db.sqlite")


(19, 1, 'Dress', 'Adidas', "Men's Fashion", 40, 1.0431592108361825, 'Black', 'XL')
(97, 2, 'Shoes', 'H&M', "Women's Fashion", 82, 4.026416271141911, 'Black', 'L')
(25, 3, 'Dress', 'Adidas', "Women's Fashion", 44, 3.337937559377053, 'Yellow', 'XL')
(57, 4, 'Shoes', 'Zara', "Men's Fashion", 23, 1.0495229563128543, 'White', 'S')
(79, 5, 'T-shirt', 'Adidas', "Men's Fashion", 79, 4.302773408398684, 'Black', 'M')
(98, 6, 'Dress', 'Adidas', "Men's Fashion", 47, 1.3795657395330458, 'Yellow', 'L')
(16, 7, 'Jeans', 'Gucci', "Men's Fashion", 37, 1.3567503746842564, 'White', 'XL')
(63, 8, 'Sweater', 'Zara', "Kids' Fashion", 64, 4.36030328941572, 'Blue', 'XL')
(96, 9, 'Sweater', 'H&M', "Men's Fashion", 53, 4.466181876278437, 'Green', 'XL')
(36, 10, 'T-shirt', 'Zara', "Kids' Fashion", 55, 4.093234402033421, 'White', 'XL')


In [6]:
!pip install -q langchain
!pip install -q openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.3/125.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00


In [ ]:
pip install langchain_openai

In [ ]:
pip install -U langchain-community

In [ ]:
pip install -U langchain_experimental

In [21]:
from langchain_openai import OpenAI
from langchain.sql_database import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

In [22]:
import os
os.environ['OPENAI_API_KEY'] = ''

In [23]:
input_db = SQLDatabase.from_uri('sqlite:///fashion_db.sqlite')
llm_1 = OpenAI(temperature=0)


db_agent = SQLDatabaseChain(llm = llm_1,
                            database = input_db,
                            verbose=True)

/usr/local/lib/python3.10/dist-packages/langchain_experimental/sql/base.py:77: UserWarning: Directly instantiating an SQLDatabaseChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [24]:
db_agent.run("how many rows are there?")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SQLDatabaseChain chain...
how many rows are there?
SQLQuery:SELECT COUNT(*) FROM fashion_products
SQLResult: [(1000,)]
Answer:1000
> Finished chain.


'1000'

In [25]:
db_agent.run("how many entries of Adidas are present?")



> Entering new SQLDatabaseChain chain...
how many entries of Adidas are present?
SQLQuery:SELECT COUNT(*) FROM fashion_products WHERE brand = "Adidas"
SQLResult: [(198,)]
Answer:198
> Finished chain.


'198'

In [26]:
db_agent.run("how many XL products of Nike are there that have a rating of more than 4?")



> Entering new SQLDatabaseChain chain...
how many XL products of Nike are there that have a rating of more than 4?
SQLQuery:SELECT COUNT(*) FROM fashion_products WHERE brand = "Nike" AND size = "XL" AND "Rating" > 4
SQLResult: [(12,)]
Answer:There are 12 XL products of Nike that have a rating of more than 4.
> Finished chain.


'There are 12 XL products of Nike that have a rating of more than 4.'